<a href="https://colab.research.google.com/github/sivasaiyadav8143/Python/blob/master/Decorators_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Decorators (Part 1)

A decorator takes in a function, adds some functionality and returns it

The decorator function adds some new functionality to the original function. This is similar to packing a gift. The decorator acts as a wrapper. The nature of the object that got decorated (actual gift inside) does not alter. But now, it looks pretty (since it got decorated).

Recall the example in the last section where we wrote a simple closure to count how many times a function had been run:

In [ ]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

In [ ]:
def add(a, b=0):
    """
    returns the sum of a and b
    """
    return a + b

In [ ]:
help(add)

Help on function add in module __main__:

add(a, b=0)
    returns the sum of a and b



Here's the memory address that `add` points to:

In [ ]:
id(add)

2352389334696

Now we create a closure using the `add` function as an argument to the `counter` function:

In [ ]:
add = counter(add)

And you'll note that `add` is no longer the same function as before. Indeed the memory address `add` points to is no longer the same:

In [ ]:
id(add)

2352404346128

In [ ]:
add(1, 2)

Function add was called 1 times


3

In [ ]:
add(2, 2)

Function add was called 2 times


4

What happened is that we put our **add** function 'through' the **counter** function - we usually say that we **decorated** our function **add**.

And we call that **counter** function a **decorator**.

There is a shorthand way of decorating our function without having to type:

``func = counter(func)``

In [ ]:
@counter
def mult(a: float, b: float=1, c: float=1) -> float:
    """
    returns the product of a, b, and c
    """
    return a * b * c

In [ ]:
mult(1, 2, 3)

Function mult was called 1 times


6

In [ ]:
mult(2, 2, 2)

Function mult was called 2 times


8

Let's do a little bit of introspection on our two decorated functions:

In [ ]:
add.__name__

'inner'

In [ ]:
mult.__name__

'inner'

As you can see, the name of the function is no longer **add** or **mult**, but instead it is the name of that **inner** function in our decorator.

In [ ]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [ ]:
help(mult)

Help on function inner in module __main__:

inner(*args, **kwargs)



As you can see, we've also lost our docstring and parameter annotations!

What about introspecting the parameters of **add** and **mult**:

In [ ]:
import inspect

In [ ]:
inspect.getsource(add)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print('Function {0} was called {1} times'.format(fn.__name__, count))\n        return fn(*args, **kwargs)\n"

In [ ]:
inspect.getsource(mult)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print('Function {0} was called {1} times'.format(fn.__name__, count))\n        return fn(*args, **kwargs)\n"

Even the signature is gone:

In [ ]:
inspect.signature(add)

<Signature (*args, **kwargs)>

In [ ]:
inspect.signature(mult)

<Signature (*args, **kwargs)>

Even the parameter defaults documentation is are gone:

In [ ]:
inspect.signature(add).parameters

mappingproxy({'args': <Parameter "*args">, 'kwargs': <Parameter "**kwargs">})

In general, when we create decorated functions, we end up "losing" a lot of the metadata of our original function!

However, we **can** put that information back in - it can get quite complicated.

Let's see how we might be able to do that for some simple things, like the docstring and the function name.

In [ ]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    return inner

In [ ]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [ ]:
help(add)

Help on function add in module __main__:

add(*args, **kwargs)
    returns sum of two integers



In [ ]:
add.__name__

'add'

At least we have the docstring and function name back... But what about the parameters? Our real **add** function takes two positional parameters, but because the closure used a generic way of accepting **\*args** and **\*\*kwargs**, we lose this information

We can use a special function in the **functools** module, called **wraps**. In fact, that function is a decorator itself!

In [ ]:
from functools import wraps

In [ ]:
def counter(fn):
    count = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))

    return inner

We specify @wraps(fn) coz wraps does not know where to get the metadata from. SO , we should specify it (hay go to fn(), copy metadata and overwrite it on closure.

In [ ]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [ ]:
help(add)

Help on function add in module __main__:

add(a:int, b:int=10) -> int
    returns sum of two integers



Yay!!! Everything is back to normal.

In [ ]:
inspect.getsource(add)

'@counter\ndef add(a: int, b: int=10) -> int:\n    """\n    returns sum of two integers\n    """\n    return a + b\n'

In [ ]:
inspect.signature(add)

<Signature (a:int, b:int=10) -> int>

In [ ]:
inspect.signature(add).parameters

mappingproxy({'a': <Parameter "a:int">, 'b': <Parameter "b:int=10">})